In [1]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, jaccard_score, f1_score, precision_score, recall_score
from skimage.metrics import structural_similarity as ssim
from sklearn import preprocessing
from keras.utils import to_categorical
from keras import regularizers
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Dropout, Lambda, GlobalAveragePooling2D
from keras.src.legacy.preprocessing.image import ImageDataGenerator

import random
import re
import glob
from tqdm import tqdm
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time
import gc

2025-05-06 20:25:35.724550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-06 20:25:35.794413: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-06 20:25:35.839787: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 20:25:36.687806: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs disponíveis: {gpus}")
else:
    print("Nenhuma GPU encontrada.")

GPUs disponíveis: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1746573952.304282     991 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746573954.221796     991 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746573954.221947     991 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
imgs_path = 'DATASET_dicom_fourier_spectrum'

TEST_SIZE = 0.1
VALIDATION_SIZE = 0.1

NUM_FOLD = 5

EPOCHS = 200
BATCH_SIZE = 32
RANDOM_STATE = 53

HIGH_FREQ_THRESHOLD = 1

output_dir = f"Results/Turnstile_CNN results ({HIGH_FREQ_THRESHOLD}%)/"
os.makedirs(output_dir, exist_ok=True)

In [4]:
def get_next_filename(output_folder, base_name, type):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    image_index = 0
    
    while True:
        output_filename = f"{base_name}_{image_index}.{type}"
        output_path = os.path.join(output_folder, output_filename)
        
        if not os.path.exists(output_path):
            return output_filename
            
        image_index += 1

In [5]:
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(0)) if match else 0

In [6]:
def sort_files_numerically(file_paths):
    return sorted(file_paths, key=lambda x: extract_number(x))

In [7]:
def read_dataset(path, jpg, png):
    print(f'Reading dataset...\n')
    
    img_type = []
    images = []
    image_paths = []

    if jpg:
        img_type.append('*.jpg')
    if png:
        img_type.append('*.png')

    print(f'Reading images from: {path}')
    
    for img_type_pattern in img_type:
        img_paths = glob.glob(os.path.join(path, img_type_pattern))
        img_paths = sort_files_numerically(img_paths)
        
        for img_path in img_paths:
            img = cv2.imread(img_path, 0)
            images.append(img)
            image_paths.append(img_path)

    images = np.array(images)
    image_paths = np.array(image_paths)

    return images, image_paths

In [8]:
all_X, all_image_paths = read_dataset(path=imgs_path, jpg=True, png=True)

Reading dataset...

Reading images from: DATASET_dicom_fourier_spectrum


In [9]:
def noise(image, threshold, add_noise, fshift):
    if add_noise:
        amplification_factor = 0.5
        rows, cols = image.shape
        crow, ccol = rows//2, cols//2

        corner = np.random.randint(0, 4)
        
        mask = np.zeros((rows, cols), dtype=bool)
        
        if corner == 0:    # Canto superior esquerdo
            mask[:crow - threshold, :ccol - threshold] = True
        elif corner == 1:  # Canto superior direito
            mask[:crow - threshold, ccol + threshold:] = True
        elif corner == 2:  # Canto inferior esquerdo
            mask[crow + threshold:, :ccol - threshold] = True
        else:              # Canto inferior direito
            mask[crow + threshold:, ccol + threshold:] = True

        fshift[mask] *= amplification_factor

    magnitude_spectrum_high = 20 * np.log(np.abs(fshift) + 1)

    return magnitude_spectrum_high

# Abordagem 1

all_specs = []
all_labels = []

half = len(all_X) // 2

fshift_dir = "DATASET_dicom_fourier_shift"
file_list = sorted(os.listdir(fshift_dir))

for i, img in enumerate(all_X):
    filename = file_list[i]
    fshift = np.load(os.path.join(fshift_dir, filename))
    
    if i < half:
        all_specs.append(noise(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=False, fshift=fshift))
        all_labels.append(0)

    else:
        all_specs.append(noise(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=True, fshift=fshift))
        all_labels.append(1)

all_specs = np.array(all_specs)
all_labels = np.array(all_labels)

# Abordagem 2

normal_specs = []
normal_labels = []

hacked_specs = []
hacked_labels = []

all_specs = []
all_labels = []

half = len(all_X) // 2

fshift_dir = "DATASET_dicom_fourier_shift"
file_list = sorted(os.listdir(fshift_dir))

for i in range(half):
    img = all_X[i]
    filename = file_list[i]
    fshift = np.load(os.path.join(fshift_dir, filename))
    
    normal_specs.append(noise(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=False, fshift=fshift))
    normal_labels.append(0)

normal_specs = np.array(normal_specs)
normal_labels = np.array(normal_labels)

for i in range(half, len(all_X)):
    img = all_X[i]
    filename = file_list[i]
    fshift = np.load(os.path.join(fshift_dir, filename))
    
    hacked_specs.append(noise(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=True, fshift=fshift))
    hacked_labels.append(1)

hacked_specs = np.array(hacked_specs)
hacked_labels = np.array(hacked_labels)

all_specs = np.concatenate([normal_specs, hacked_specs])
all_labels = np.concatenate([normal_labels, hacked_labels])

# Abordagem 3

all_specs = []
all_labels = []

half = len(all_X) // 2

fshift_dir = "DATASET_dicom_fourier_shift"
file_list = sorted(os.listdir(fshift_dir))

temp_dir = "temp_processed"
os.makedirs(temp_dir, exist_ok=True)

for i in range(len(all_X)):
    img = all_X[i]
    filename = file_list[i]
    fshift = np.load(os.path.join(fshift_dir, filename))
    
    if i < half:
        processed = noise(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=False, fshift=fshift)
        label = 0
    else:
        processed = noise(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=True, fshift=fshift)
        label = 1

    np.save(os.path.join(temp_dir, f"spec_{i}.npy"), processed)
    np.save(os.path.join(temp_dir, f"label_{i}.npy"), label)

for i in range(len(all_X)):
    spec = np.load(os.path.join(temp_dir, f"spec_{i}.npy"))
    label = np.load(os.path.join(temp_dir, f"label_{i}.npy"))
    all_specs.append(spec)
    all_labels.append(label)

all_specs = np.array(all_specs)
all_labels = np.array(all_labels)

import shutil

shutil.rmtree(temp_dir)

# Abordagem 4

In [10]:
def process_and_save_chunks(chunk_size=50):
    half = len(all_X) // 2

    fshift_dir = "DATASET_dicom_fourier_shift"
    file_list = sorted(os.listdir(fshift_dir))
    temp_dir = "temp_processed"
    os.makedirs(temp_dir, exist_ok=True)

    for chunk_start in tqdm(range(0, len(all_X), chunk_size)):
        chunk_end = min(chunk_start + chunk_size, len(all_X))
        
        specs_chunk = []
        labels_chunk = []
        
        for i in range(chunk_start, chunk_end):
            img = all_X[i]
            filename = file_list[i]
            fshift = np.load(os.path.join(fshift_dir, filename))
            
            if i < half:
                processed = noise(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=False, fshift=fshift)
                label = 0
            else:
                processed = noise(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=True, fshift=fshift)
                label = 1
            
            specs_chunk.append(processed)
            labels_chunk.append(label)
        
        np.save(os.path.join(temp_dir, f"specs_chunk_{chunk_start}.npy"), np.array(specs_chunk))
        np.save(os.path.join(temp_dir, f"labels_chunk_{chunk_start}.npy"), np.array(labels_chunk))
        
        del specs_chunk, labels_chunk
        gc.collect()

In [11]:
process_and_save_chunks(chunk_size=50)

100%|███████████████████████████████████████████████████████████████████████████████████| 52/52 [05:03<00:00,  5.83s/it]


In [12]:
class LazyDataLoader:
    def __init__(self, temp_dir="temp_processed"):
        self.temp_dir = temp_dir
        self.spec_chunks = sorted(glob.glob(os.path.join(temp_dir, "specs_chunk_*.npy")), 
                               key=lambda x: int(re.findall(r'specs_chunk_(\d+).npy', x)[0]))
        self.label_chunks = sorted(glob.glob(os.path.join(temp_dir, "labels_chunk_*.npy")), 
                                key=lambda x: int(re.findall(r'labels_chunk_(\d+).npy', x)[0]))
        
    def get_total_samples(self):
        total = 0
        for spec_file in self.spec_chunks:
            total += np.load(spec_file).shape[0]
        return total

    def load_labels(self):
        all_labels = []
        for label_file in self.label_chunks:
            all_labels.extend(np.load(label_file))
        return np.array(all_labels)

In [13]:
lazy_loader = LazyDataLoader()
all_indices = np.arange(lazy_loader.get_total_samples())
all_labels = lazy_loader.load_labels()

In [14]:
print("Quantidade das imagens:", all_indices.shape)
print("Exemplo dos labels (False = original, True = com ruído):", all_labels)

Quantidade das imagens: (2581,)
Exemplo dos labels (False = original, True = com ruído): [0 0 0 ... 1 1 1]


In [15]:
X_train, X_test, y_train, y_test, = train_test_split(
        all_indices, all_labels,
        test_size=TEST_SIZE, 
        stratify=all_labels,
#       random_state=RANDOM_STATE
    )

# Plotar imagens caso necessário

def plot_specs(specs, title):
    total_images = len(specs)
    imgs_per_figure = 100
    cols = 5
    rows = (imgs_per_figure + cols - 1) // cols

    for start in range(0, total_images, imgs_per_figure):
        plt.figure(figsize=(cols * 3, rows * 3))
        end = min(start + imgs_per_figure, total_images)
        
        for i in range(start, end):
            plt.subplot(rows, cols, i - start + 1)
            plt.imshow(specs[i], cmap='gray')
            plt.title(f"{title} - freq spec: img {i+1}")
            plt.axis('off')
            
        plt.tight_layout()
        plt.show()

plot_specs(X_test, title="test")

In [16]:
#pausa de teste

In [17]:
# Define as layers do modelo ResNet50
def model_resnet50():
# Camada com tamanho padronizado para imagens em escala de cinza
    inputs = Input(shape=(512, 512, 1))

# 1. Conversão para 3 canais
    x = Lambda(
        lambda x: tf.stack([x[..., 0]]*3, axis=-1),
        output_shape=(512, 512, 3)
    )(inputs)

# 2. Pré-processamento ResNet
    x = tf.keras.applications.resnet.preprocess_input(x)

# 3. Carregar ResNet-50
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_tensor=x
    )

# 4. Topo personalizado
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    x = Dense(150, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
    x = Dropout(0.25)(x)

    x = Dense(100, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
    x = Dropout(0.25)(x)

    outputs = Dense(2, activation='softmax')(x)

# Compilação
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer='Adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Abordagem 1

acc = []
jacc = []
f1 = []
prec = []
rec = []

# Configurar K-Fold com random_state fixo para reprodutibilidade
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Verificar quais folds já foram completados
completed_folds = []
for f in range(1, 6):
    model_path = output_dir + f'model_fold_{f}.keras'
    if os.path.exists(model_path):
        completed_folds.append(f)
print(f"Folds concluídos: {completed_folds}")

fold_no = 1
histories = []
metrics = []

for train_idx, val_idx in kfold.split(X_train, y_train):
    # Pular folds já concluídos
    if fold_no in completed_folds:
        print(f"\nPulando fold {fold_no} (já concluído)")
        fold_no += 1
        continue

    print(f'\nTreinando Fold {fold_no}/5')
    
    # Split dos dados
    X_train_fold = X_train[train_idx]
    y_train_fold = y_train[train_idx]
    X_val_fold = X_train[val_idx]
    y_val_fold = y_train[val_idx]

    # Pré-processamento final
    X_train_fold = np.expand_dims(X_train_fold, axis=-1)
    X_val_fold = np.expand_dims(X_val_fold, axis=-1)
    y_train_fold_cat = to_categorical(y_train_fold, 2)
    y_val_fold_cat = to_categorical(y_val_fold, 2)

    # Criar novo modelo para cada fold
    model = model_resnet50()

    # Checkpoint com nome do fold
    checkpoint_filepath = output_dir + f'model_fold_{fold_no}.keras'
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=20, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=False,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]

    # Calcula tempo (start)
    start_time = time.time()
    
    # Treinar modelo
    history = model.fit(
        X_train_fold, y_train_fold_cat,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val_fold, y_val_fold_cat),
        callbacks=callbacks,
        verbose=1
    )

    # Calcula tempo (end)
    end_time = time.time()

    training_time = end_time - start_time
    print(f"\nO modelo demorou {training_time:.2f} segundos para treinar.")

    # Coletar métricas e salvar modelo
    predictions = model.predict(X_val_fold)
    y_pred = np.argmax(predictions, axis=1)
    
    metrics.append({
        'fold': fold_no,
        'report': classification_report(y_val_fold, y_pred, output_dict=True, zero_division=0),
        'matrix': confusion_matrix(y_val_fold, y_pred)
    })

    # Métricas de classificação (por fold)
    acc.append(accuracy_score(y_val_fold, y_pred))
    jacc.append(jaccard_score(y_val_fold, y_pred))
    f1.append(f1_score(y_val_fold, y_pred))
    prec.append(precision_score(y_val_fold, y_pred))
    rec.append(recall_score(y_val_fold, y_pred))

    # Salvar métricas em um arquivo .txt
    metrics_filename = os.path.join(output_dir, f'metrics_fold_{fold_no}.txt')
    with open(metrics_filename, 'w') as f:
        f.write(f"Fold {fold_no} Metrics:\n")
        f.write(f"Accuracy: {acc[-1]}\n")
        f.write(f"Jaccard Score: {jacc[-1]}\n")
        f.write(f"F1 Score: {f1[-1]}\n")
        f.write(f"Precision: {prec[-1]}\n")
        f.write(f"Recall: {rec[-1]}\n")
        f.write("\nClassification Report:\n")
        f.write(classification_report(y_val_fold, y_pred, zero_division=0))
        f.write("\nConfusion Matrix:\n")
        f.write(np.array2string(confusion_matrix(y_val_fold, y_pred)))

    # Limpeza de memória
    del model
    tf.keras.backend.clear_session()
    gc.collect()

    # Salvar checkpoint a cada 3 folds
    if fold_no % 3 == 0:
        print(f"\nCheckpoint: Folds {fold_no-2}-{fold_no} concluídos")

    fold_no += 1

# Abordagem 2

class LazyDataset:
    def __init__(self, temp_dir):
        self.spec_files = sorted(glob.glob(os.path.join(temp_dir, "specs_chunk_*.npy")), 
                         key=lambda x: int(re.search(r'_(\d+)\.npy', x).group(1)))
        self.label_files = sorted(glob.glob(os.path.join(temp_dir, "labels_chunk_*.npy")), 
                          key=lambda x: int(re.search(r'_(\d+)\.npy', x).group(1)))
        self.total_samples = sum(len(np.load(f)) for f in self.spec_files)
        self.chunk_size = len(np.load(self.spec_files[0])) if self.spec_files else 0

    def __len__(self):
        return self.total_samples

    def create_lazy_dataset(self, indices, batch_size=BATCH_SIZE):
        def generator():
            from collections import defaultdict # Import dentro do escopo do generator

            # Agrupa índices por chunk para minimizar acesso ao disco
            chunk_map = defaultdict(list)
            for idx in indices:
                chunk_idx = idx // self.chunk_size
                chunk_map[chunk_idx].append(idx % self.chunk_size)
            
            # Processa chunks em grupos
            for chunk_idx, local_indices in chunk_map.items():
                specs = np.load(self.spec_files[chunk_idx])
                labels = np.load(self.label_files[chunk_idx])
                
                # Embaralha os índices locais para aumentar aleatoriedade
                np.random.shuffle(local_indices)
                
                for i in range(0, len(local_indices), batch_size):
                    batch_indices = local_indices[i:i + int(batch_size)]
                    batch_specs = specs[batch_indices]
                    batch_labels = labels[batch_indices]
                    
                    # Adiciona dimensão do canal e converte labels
                    yield np.expand_dims(batch_specs, axis=-1), to_categorical(batch_labels, 2)
        
        return generator

# Abordagem 3

In [18]:
class LazyDataset:
    def __init__(self, temp_dir):
        self.spec_files = sorted(glob.glob(os.path.join(temp_dir, "specs_chunk_*.npy")), 
                         key=lambda x: int(re.search(r'_(\d+)\.npy', x).group(1)))
        self.label_files = sorted(glob.glob(os.path.join(temp_dir, "labels_chunk_*.npy")), 
                          key=lambda x: int(re.search(r'_(\d+)\.npy', x).group(1)))
        
        # Carrega o tamanho real de cada chunk
        self.chunk_sizes = [np.load(f).shape[0] for f in self.spec_files]
        self.cumulative_sizes = np.cumsum(self.chunk_sizes)
        self.total_samples = sum(self.chunk_sizes)

    def __len__(self):
        return self.total_samples

    def get_chunk_index(self, global_idx):
        chunk_idx = np.searchsorted(self.cumulative_sizes, global_idx, side='right')
        return chunk_idx

    def create_lazy_dataset(self, indices, batch_size=BATCH_SIZE):
        def generator():
            from collections import defaultdict

            # Agrupa índices por chunk com base nos tamanhos reais
            chunk_map = defaultdict(list)
            for idx in indices:
                chunk_idx = self.get_chunk_index(idx)
                chunk_map[chunk_idx].append(idx - (self.cumulative_sizes[chunk_idx-1] if chunk_idx > 0 else 0))
            
            # Processa cada chunk
            for chunk_idx, local_indices in chunk_map.items():
                specs = np.load(self.spec_files[chunk_idx])
                labels = np.load(self.label_files[chunk_idx])
                
                # Garante o processamento na ordem original
                for i in range(0, len(local_indices), batch_size):
                    batch_indices = local_indices[i:i + batch_size]
                    batch_specs = specs[batch_indices]
                    batch_labels = labels[batch_indices]
                    
                    yield np.expand_dims(batch_specs, axis=-1), to_categorical(batch_labels, 2)
        
        return generator

In [ ]:
lazy_data = LazyDataset("temp_processed")
acc, jacc, f1, prec, rec = [], [], [], [], []

# Gera índices completos para o KFold
full_indices = np.arange(len(lazy_data))
all_labels = np.concatenate([np.load(f) for f in lazy_data.label_files])

kfold = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True, random_state=RANDOM_STATE)

completed_folds = []
for f in range(1, NUM_FOLD + 1):
    model_path = output_dir + f'model_fold_{f}.keras'
    if os.path.exists(model_path):
        completed_folds.append(f)
print(f"Folds concluídos: {completed_folds}")

current_fold = 1
histories = []
metrics = []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(full_indices, all_labels)):
    # Pular folds já concluídos
    if current_fold in completed_folds:
        print(f"\nPulando fold {current_fold} (já concluído)")
        current_fold += 1
        continue

    print(f'\nTreinando Fold {current_fold}/{NUM_FOLD}')

    # Cria novo modelo
    model = model_resnet50()

    # Checkpoint com nome do fold
    checkpoint_filepath = output_dir + f'model_fold_{current_fold}.keras'
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=20, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=False,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]
    
    # Cria datasets
    train_gen = lazy_data.create_lazy_dataset(train_idx, BATCH_SIZE)
    val_gen = lazy_data.create_lazy_dataset(val_idx, BATCH_SIZE)
    
    train_dataset = tf.data.Dataset.from_generator(
        train_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, 512, 512, 1), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 2), dtype=tf.float32)
        )
    ).prefetch(tf.data.AUTOTUNE)
    
    val_dataset = tf.data.Dataset.from_generator(
        val_gen,
        output_signature=(
            tf.TensorSpec(shape=(None, 512, 512, 1), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 2), dtype=tf.float32)
        )
    ).prefetch(tf.data.AUTOTUNE)

    # Calcula tempo (start)
    start_time = time.time()
    
    # Treinamento
    history = model.fit(
        train_dataset,
        steps_per_epoch=int(np.ceil(len(train_idx)/BATCH_SIZE)),
        validation_data=val_dataset,
        validation_steps=int(np.ceil(len(val_idx)/BATCH_SIZE)),
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=1
    )

    # Calcula tempo (end)
    end_time = time.time()

    training_time = end_time - start_time
    print(f"\nO modelo demorou {training_time:.2f} segundos para treinar.")
    
    # Validação
    y_val_true = all_labels[val_idx]
    y_pred = []
    
    for batch in val_dataset:
        preds = model.predict(batch[0], verbose=0)
        y_pred.extend(np.argmax(preds, axis=1))

    y_pred = np.array(y_pred)[:len(val_idx)]

    # Verificação de consistência
    assert len(y_val_true) == len(y_pred), f"Dimensões inconsistentes: {len(y_val_true)} vs {len(y_pred)}"

    # Cálculo de métricas
    metrics.append({
        'fold': current_fold,
        'report': classification_report(y_val_true, y_pred, output_dict=True, zero_division=0),
        'matrix': confusion_matrix(y_val_true, y_pred)
    })

    acc.append(accuracy_score(y_val_true, y_pred))
    jacc.append(jaccard_score(y_val_true, y_pred))
    f1.append(f1_score(y_val_true, y_pred))
    prec.append(precision_score(y_val_true, y_pred))
    rec.append(recall_score(y_val_true, y_pred))

    # Salvar métricas em um arquivo .txt
    metrics_filename = os.path.join(output_dir, f'metrics_fold_{current_fold}.txt')
    with open(metrics_filename, 'w') as f:
        f.write(f"Fold {current_fold} Metrics:\n")
        f.write(f"Acurácia: {acc[-1]:.4f}\n")
        f.write(f"Jaccard: {jacc[-1]:.4f}\n")
        f.write(f"F1-Score: {f1[-1]:.4f}\n")
        f.write(f"Precisão: {prec[-1]:.4f}\n")
        f.write(f"Recall: {rec[-1]:.4f}\n")
        f.write("\nClassification Report:\n")
        f.write(classification_report(y_val_true, y_pred, zero_division=0))
        f.write("\nConfusion Matrix:\n")
        f.write(np.array2string(confusion_matrix(y_val_true, y_pred)))

    current_fold += 1
    
    # Limpeza de memória
    del model
    tf.keras.backend.clear_session()
    gc.collect()

Folds concluídos: [1]

Pulando fold 1 (já concluído)

Treinando Fold 2/5


I0000 00:00:1746574368.225246     991 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746574368.225362     991 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746574368.225420     991 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746574368.731993     991 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746574368.732095     991 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-06

Epoch 1/200


I0000 00:00:1746574396.716156    1152 service.cc:146] XLA service 0x7f28980033b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746574396.716202    1152 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-05-06 20:33:17.678047: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-05-06 20:33:20.394857: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101
2025-05-06 20:33:24.173565: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_15567', 196 bytes spill stores, 196 bytes spill loads

2025-05-06 20:33:44.429928: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng51{k2=0,k13=2,k14=3} for conv (f32[32,256,128,128]

 1/65 ━━━━━━━━━━━━━━━━━━━━ 1:22:55 78s/step - accuracy: 0.6875 - loss: 4.5703

...


Epoch 6: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 40s 620ms/step - accuracy: 0.4447 - loss: 0.8865 - val_accuracy: 0.4641 - val_loss: 0.9222
Epoch 7/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1994 - loss: 1.2719
Epoch 7: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.2020 - loss: 1.2697 - val_accuracy: 0.0000e+00 - val_loss: 0.8874
Epoch 8/200
38/65 ━━━━━━━━━━━━━━━━━━━━ 11s 425ms/step - accuracy: 0.4555 - loss: 0.8640
Epoch 8: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 16s 247ms/step - accuracy: 0.5897 - loss: 0.8402 - val_accuracy: 1.0000 - val_loss: 0.8119
Epoch 9/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 768ms/step - accuracy: 0.1013 - loss: 0.9991
Epoch 9: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.1033 - loss: 0.9975 - val_accuracy: 1.0000 - val_loss: 0.8057
Epoch 10/200
38/65 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0

2025-05-06 20:53:43.977907: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 528ms/step - accuracy: 0.0415 - loss: 0.7959
Epoch 13: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.0428 - loss: 0.7957 - val_accuracy: 1.0000 - val_loss: 0.7516
Epoch 14/200
38/65 ━━━━━━━━━━━━━━━━━━━━ 23s 864ms/step - accuracy: 0.3728 - loss: 0.7588
Epoch 14: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 45s 701ms/step - accuracy: 0.5168 - loss: 0.7546 - val_accuracy: 0.4641 - val_loss: 0.7566
Epoch 15/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 845ms/step - accuracy: 0.0415 - loss: 0.7752
Epoch 15: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - accuracy: 0.0427 - loss: 0.7751 - val_accuracy: 0.0000e+00 - val_loss: 0.7545
Epoch 16/200
38/65 ━━━━━━━━━━━━━━━━━━━━ 13s 507ms/step - accuracy: 0.3728 - loss: 0.7460
Epoch 16: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 19s 294ms/step - accuracy: 0.5168 - loss: 0.7420 - val_accuracy: 1.0000 

2025-05-06 21:02:54.817238: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]



Epoch 22: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 41s 641ms/step - accuracy: 0.5168 - loss: 0.7203 - val_accuracy: 0.4641 - val_loss: 0.7241
Epoch 23/200
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 528ms/step - accuracy: 0.0418 - loss: 0.7431
Epoch 23: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 63s 752ms/step - accuracy: 0.0431 - loss: 0.7429 - val_accuracy: 0.0000e+00 - val_loss: 0.7271
Epoch 24/200
38/65 ━━━━━━━━━━━━━━━━━━━━ 11s 435ms/step - accuracy: 0.3728 - loss: 0.7194
Epoch 24: val_accuracy did not improve from 1.00000
65/65 ━━━━━━━━━━━━━━━━━━━━ 16s 253ms/step - accuracy: 0.5168 - loss: 0.7158 - val_accuracy: 1.0000 - val_loss: 0.6915

O modelo demorou 1893.06 segundos para treinar.

Treinando Fold 3/5
Epoch 1/200


In [ ]:
# Limpeza final
shutil.rmtree("temp_processed", ignore_errors=True)
tf.keras.backend.clear_session()
gc.collect()